In [1]:
# Import the required libraries

import torch
import os
import torchvision.datasets as datasets
import torchvision
from torch.utils.data import Dataset, DataLoader, random_split
import torch.utils.data
from torchvision import transforms
from torch.nn.functional import interpolate
import torch.nn as nn
from torch.autograd import Variable
import math
import numpy as np
from natsort import natsorted
from PIL import Image
import random

In [ ]:
# Code to upload dataset to google colab

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import zipfile

path_to_zip_file = 'drive/MyDrive/data.zip'
directory_to_extract_to = 'data/'
with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall(directory_to_extract_to)

In [2]:
class VISCHEMA_PLUS(Dataset):
    def __init__(self, image_dir = 'VISCHEMA_PLUS/images', label_dir = 'VISCHEMA_PLUS/vms', transform = None):
        self.transform = transform
        
        self.image_dir = image_dir
        self.label_dir = label_dir
        
        all_images = os.listdir(image_dir)
        self.all_images = natsorted(all_images)
        
    def __len__(self):
        return len(self.all_images)

    def __getitem__(self, idx):    
        convert_tensor = transforms.ToTensor()
        
        image = Image.open(f"{self.image_dir}/{self.all_images[idx]}").convert("RGB")
        image = convert_tensor(image)

        label = Image.open(f"{self.label_dir}/{self.all_images[idx]}").convert("RGB")
        label = convert_tensor(label)
        
        if self.transform != None:
            image = self.transform(image)
            label = self.transform(label)
            
        return image, label

# I have resized the images to 512x512 as this will make designing the model easier.
image_transforms = transforms.Compose([
    transforms.Resize(128),
    transforms.Normalize(0.5,0.5)
])
        
dataset = VISCHEMA_PLUS(transform = image_transforms)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

batch_size = 2
train_loader = DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle=True)
test_loader = DataLoader(dataset = test_dataset, batch_size = batch_size, shuffle=True)

print(f'{len(train_dataset)} Items in Train dataset')
print(f'{len(test_dataset)}  Items in Test dataset')

1280 Items in Train dataset
320  Items in Test dataset


In [ ]:
import matplotlib.pyplot as plt

figure = plt.figure(figsize=(15,15))
cols, rows = 3, 4
for i in range(rows):
    
    images, labels = next(iter(train_loader))
    
    # The image
    figure.add_subplot(rows,cols, cols*i+1)
    plt.axis("off")
    plt.imshow((images[0].squeeze().permute(1, 2, 0) +1 )/2 )
    
    # The Label
    figure.add_subplot(rows,cols, cols*i+2)
    plt.axis("off")
    plt.imshow((labels[0].squeeze().permute(1, 2, 0) +1 )/2)
    
plt.tight_layout()
plt.show()

In [3]:
class Block(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.main = nn.Sequential(
            nn.Conv2d(in_channels,out_channels,3,padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(out_channels),
            nn.Conv2d(out_channels,out_channels,3,padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(out_channels),
        )
    def forward(self, x):
        return self.main(x)
    
class Encoder(nn.Module):
    def __init__(self, channels=(3,64,128,256,512)):
        super().__init__()
        self.encoding_blocks = nn.ModuleList(
            [Block(channels[i], channels[i+1]) for i in range(len(channels)-1)]
        )
        self.pool = nn.MaxPool2d(2)
    
    def forward(self, x):
        features = []
        
        for block in self.encoding_blocks:
            x = block(x)
            features.append(x)
            x = self.pool(x)
                 
        return features
        
class Decoder(nn.Module):
    def __init__(self, channels=(512,256,128,64)):
        super().__init__()
        self.channels = channels
        self.upconvs = nn.ModuleList([nn.ConvTranspose2d(self.channels[i], self.channels[i+1],2,2) for i in range(len(channels)-1)])
        self.decoder_blocks = nn.ModuleList(
            [Block(channels[i], channels[i+1]) for i in range(len(channels)-1)]
        )
    
    def forward(self, x, encoder_features):
        for i in range(len(self.channels)-1):
            x = self.upconvs[i](x)
            features = self.crop(encoder_features[i], x)
            x = torch.cat([x, features], dim=1)
            x = self.decoder_blocks[i](x)
        return x
    
    def crop(self, features, x):
        _, _, height, width = x.shape
        features = transforms.CenterCrop([height, width])(features)
        return features
        
class Generator(nn.Module):
    def __init__(self, 
                 encode_channels=(3,64,128,256,512), 
                 decode_channels=(512,256,128,64), 
                 num_class=3, 
                 retain_dim=True, 
                 output_size=(572,572)):
        
        super(Generator, self).__init__()
        
        self.encoder = Encoder(encode_channels)
        self.decoder = Decoder(decode_channels)
        
        self.head = nn.Conv2d(decode_channels[-1], num_class, 1)
        
        """# Initialize weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.ConvTranspose2d):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)"""

    def forward(self, x):
        
        encoding_features = self.encoder(x)
        decoding_features = self.decoder(encoding_features[::-1][0], encoding_features[::-1][1:])
        output = self.head(decoding_features)
        output = nn.Tanh()(output)
        
        return output

model = Generator()

total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'{total_params} Parameters in UNet Generator')

images, _ = next(iter(train_loader))
output = model(images)
print(output.shape)

del model, output, images

7703107 Parameters in UNet Generator
torch.Size([2, 3, 128, 128])


In [4]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        
        def block(in_channels, out_channels, norm=True):
            layers = [nn.Conv2d(in_channels, out_channels, 4,stride=2, padding=1)]
            if norm:
                layers.append(nn.BatchNorm2d(out_channels))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers
        
        self.model = nn.Sequential(
            *block(6, 64, norm = False),
            *block(64, 128),
            *block(128,256),
            *block(256,512),
            *block(512,512),
            nn.ZeroPad2d((1,0,1,0)),
            nn.Conv2d(512,1,4,padding=1, bias=False)
        )
        
    def forward(self, images, labels):
        
        image_input = torch.cat((images, labels), 1)
        return self.model(image_input)

model = Discriminator()

total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'{total_params} Parameters in Discriminator')

images, labels = next(iter(train_loader))
output = model(images, labels)
print(output.shape)

del model, output, images

6965440 Parameters in Discriminator
torch.Size([2, 1, 4, 4])


In [8]:
# Set up our training environment
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
patch = (1, 4, 4)

net_G = Generator()
net_D = Discriminator()

net_G, net_D = net_G.to(device), net_D.to(device)

criterion_GAN = nn.MSELoss()
criterion_pixelwise = nn.L1Loss() 

# Loss weight of L1 pixel-wise loss between translated image and real image
# This value is copied from the PIX2PIX architecture
lambda_pixel = 100

optim_G = torch.optim.Adam(net_G.parameters(), lr=0.0001, betas = (0,0.9))
optim_D = torch.optim.Adam(net_D.parameters(), lr=0.0001, betas = (0,0.9))
num_epochs = 250

Starting Training
Testing


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 8.00 GiB total capacity; 7.27 GiB already allocated; 0 bytes free; 7.29 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
torch.save(net_D.state_dict(), f'weights-NETD-1-{epoch}.pkl')
torch.save(net_G.state_dict(), f'weights-NETG-1-{epoch}.pkl')

In [ ]:
# Visualise Output

net_G = Generator()
net_G.load_state_dict(torch.load("weights-NETG-1-48.pkl", map_location=torch.device('cpu')), strict=False)

import matplotlib.pyplot as plt

images, labels = next(iter(train_loader))
figure = plt.figure(figsize=(15,15))
output = net_G(images)

cols, rows = 3, 4
for i in range(rows):

    figure.add_subplot(rows,cols, cols*i+1)
    plt.axis("off")
    plt.imshow((images[i].cpu().detach().squeeze().permute(1, 2, 0) +1 )/ 2 )
    
    figure.add_subplot(rows,cols, cols*i+2)
    plt.axis("off")
    plt.imshow((output[i].cpu().detach().squeeze().permute(1, 2, 0) +1 )/ 2 )  

    figure.add_subplot(rows,cols, cols*i+3)
    plt.axis("off")
    plt.imshow((labels[i].cpu().detach().squeeze().permute(1, 2, 0) +1 )/ 2 )
plt.tight_layout()
plt.show()